In [1]:
import time
import os
import pandas as pd
import numpy as np
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"
test_set_complete=pd.read_csv(output_path+"general/test_set_final.csv")
train_set=pd.read_csv(output_path+"general/train_set_final.csv")
dev_set=pd.read_csv(output_path+"general/dev_set_final.csv")
df=pd.concat([test_set_complete,dev_set,train_set]).reset_index(drop=True)

In [4]:
df=df.assign(abstract_for_prompt=np.where(df.abstract_clean!="",
                                                   "Abstract: "+df.abstract_clean,""))
df=df.assign(acknowledgments_for_prompt=np.where(df.acknowledgments_clean!="",
                                                   "\nAcknowledgments: "+df.acknowledgments_clean,""))
df=df.assign(abstract_for_prompt=df.abstract_for_prompt.fillna(""))
df=df.assign(paper_text="Title: "+df.title_clean+"\n"+df.abstract_for_prompt)

In [6]:
ack_preprompt="""Identify the NLP task(s) addressed in this paper. Answer in the format [task1,...,taskn,...]"""

ack_postprompt="""The NLP tasks addressed in this paper are:\n["""

In [7]:
df=df.assign(task_prompt_text=ack_preprompt+"\n"+df.paper_text+"\n"+ack_postprompt)

In [8]:
df=df.loc[df.label==1]
df=df.loc[(~df.task_annotation.isna())|(~df.method_annotation.isna())].reset_index(drop=True)

## cost estimation

In [9]:
np.sum(df['task_prompt_text'].str.split().apply(len))

20138

In [ ]:
$0.0600 / 1K tokens

In [9]:
df.shape

(1000, 13)

In [9]:
5000*50

250000

In [ ]:
1627921

In [10]:
(((793221+250000)*1000/750)/1000)*0.0600

83.45767999999998

## requests

In [9]:
df.shape

(130, 21)

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    print(input_prompt)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("###################################")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=40,logprobs=1)
    df.loc[i,'GPT3_response']=completion.choices[0].text
    f = open("responses_f_test.txt", "a")
    f.write(d['task_prompt_text'])
    f.write(completion.choices[0].text)
    f.write("################################")
    f.write("\n")
    f.close()
    print(input_prompt)
    print(completion.choices[0].text)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("################################")

In [12]:
df=df.assign(clean_response=df.GPT3_response.replace("\n","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("\[task2\]",",",regex=True).replace("\[task3\]",",",regex=True).replace("task1\]","",regex=True).replace("1\]","",regex=True).replace("\[3\]",",",regex=True).replace("\[2\]",",",regex=True).str.rstrip(']'))
df=df.assign(clean_response=df.clean_response.str.split(","))

tasks=df.loc[:,['ID','clean_response']]

tasks=tasks.explode("clean_response")

tasks=tasks.rename(columns={'clean_response':'tasks'})

tasks=tasks.assign(tasks=tasks.tasks.str.lstrip().str.rstrip())
tasks=tasks.loc[tasks.tasks!='']

In [13]:
tasks.tasks.value_counts()

text classification       27
information extraction    12
information retrieval     11
machine translation       10
part-of-speech tagging     6
                          ..
Curriculum Learning        1
word segmentation          1
disambiguation             1
typing                     1
authorship attribution     1
Name: tasks, Length: 298, dtype: int64

In [14]:
tasks

,ID,tasks
0,wandji-tchami-grabar-2014-towards,semantic annotation
0,wandji-tchami-grabar-2014-towards,part-of-speech tagging
0,wandji-tchami-grabar-2014-towards,verb sense disambiguation
1,soh-etal-2019-legal,text classification
1,soh-etal-2019-legal,topic modeling
...,...,...
127,cavicchio-2009-modulation,annotation of blended emotions
128,khanna-etal-2022-idiap,text classification
128,khanna-etal-2022-idiap,hate speech detection
129,solorio-etal-2013-case,authorship attribution


In [15]:
df.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_f2.csv",index=False)

tasks.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_cleaned_f2.csv",index=False)